In [24]:
from selenium import webdriver
# import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

print("[INFO] Starting BOT process for get download official oils prices URL")
go_time = time.time()
chrome_options = Options()

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# decomment for see navigation in real
# chrome_options.add_argument("--headless")

# Initialise solanium driver
driver = webdriver.Chrome(options=chrome_options)
# use undetected_chromedriver if we want discretion
# driver = uc.Chrome(options=chrome_options)

print("[BOT] go to url")
WebDriverWait(driver, 10)
driver.get("https://www.ecologie.gouv.fr/politiques-publiques/prix-produits-petroliers")
time.sleep(5)

# remove popup for cookie
print("[BOT] remove popup for cookie")
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable(
        (By.CSS_SELECTOR, 'button.fr-btn[title="Accepter tous les cookies"]')
    )
).click()

time.sleep(2)

# click on 'demarré' button
print("[BOT] click on 'demarré' button")
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((
            By.CSS_SELECTOR,"a.use-ajax.fr-btn[href^='/ajax/simulator/load/customer_simulator_energies_fuel_block']"))
).click()
time.sleep(2)

# click on 'Étape suivante'' button
print("[BOT] click on 'Étape suivante'' button")
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[value='Étape suivante']"))
).click()
time.sleep(2)

print("[BOT] choose 'Carburants routiers' radio button")
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//label[.//span[contains(normalize-space(.), 'Carburants routiers')]]"))
).click()
time.sleep(2)

# click on radio "suivre l evolution"
print("[BOT] choose 'suivre l evolution' radio button")
radio = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='radio'][value='monitor']"))
)
driver.execute_script("arguments[0].click();", radio)
time.sleep(2)

# click on button for validate formulary 'étape suivante'
print("[BOT] click on button for validate formulary 'étape suivante'")
next_btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input.webform-button--next"))
)
driver.execute_script("arguments[0].click();", next_btn)
time.sleep(5)

# click on all carburants in list
print("[BOT] click on all carburants in list:")
print("gazole")
label = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for^='edit-monitor-road-fuels-gazole']"))
)
label.click()
time.sleep(2)

#full input starting data dates
print("[BOT] full input of starting data dates")
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='monitor_start_date']"))
).send_keys("01012015")
time.sleep(2)

#full input ending data dates
print("[BOT] full input of ending data dates")
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='monitor_end_date']"))
).send_keys("01012100")
time.sleep(2)

#click on button for validate formulary 'Soumettre'
print("[BOT] click on button for validate formulary 'Soumettre'")
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='submit'][value='Soumettre']"))
).click()
time.sleep(10)

# wait response and get link for download data
print("[BOT] wait response (almost 20 seconds) and get link for download data")
download_link = WebDriverWait(driver, 120).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "div.fr-col-none:last-child div.fr-download > a.fr-link--download"))
).get_attribute("href")
print("[BOT] Download link scrapping by bot:", download_link)
print("[BOT] Bot result in", time.time() - go_time)

[INFO] Starting BOT process for get download official oils prices URL
[BOT] go to url
[BOT] remove popup for cookie
[BOT] click on 'demarré' button
[BOT] click on 'Étape suivante'' button
[BOT] choose 'Carburants routiers' radio button
[BOT] choose 'suivre l evolution' radio button
[BOT] click on button for validate formulary 'étape suivante'
[BOT] click on all carburants in list:
gazole
[BOT] full input of starting data dates
[BOT] full input of ending data dates
[BOT] click on button for validate formulary 'Soumettre'
[BOT] wait response (almost 20 seconds) and get link for download data
[BOT] Download link scrapping by bot: https://www.ecologie.gouv.fr/simulator-energies/monitoring/export/937ecb6303324817b9244d477e2cf34bccc31f9c
[BOT] Bot result in 38.5638964176178


In [25]:
import pandas as pd
import os

months_fr_en = {
    'janvier':'January', 'février':'February', 'mars':'March', 'avril':'April',
    'mai':'May', 'juin':'June', 'juillet':'July', 'août':'August',
    'septembre':'September', 'octobre':'October', 'novembre':'November',
    'décembre':'December'
}

##### HTT
df_carburants_htt = pd.read_excel(
    download_link,
    sheet_name="Carburants - mensuel HTT",
    skiprows=0
)
for fr, en in months_fr_en.items():
    df_carburants_htt['Date'] = df_carburants_htt['Date'].str.replace(fr, en)
df_carburants_htt['Date'] = pd.to_datetime(df_carburants_htt['Date'], format='%Y %B')
df_carburants_htt = df_carburants_htt.rename(columns={"Gazole €/l htt": "gazole_eur_liter_htt"})
print("df_carburants_htt\n", df_carburants_htt)

##### TTC
df_carburants_ttc = pd.read_excel(
    download_link,
    sheet_name="Carburants - mensuel TTC",
    skiprows=0
)
for fr, en in months_fr_en.items():
    df_carburants_ttc['Date'] = df_carburants_ttc['Date'].str.replace(fr, en)
df_carburants_ttc['Date'] = pd.to_datetime(df_carburants_ttc['Date'], format='%Y %B')
df_carburants_ttc = df_carburants_ttc.rename(columns={"Gazole €/l ttc": "gazole_eur_liter_ttc"})
print("df_carburants_ttc\n", df_carburants_ttc)

df_merged = pd.merge(df_carburants_htt, df_carburants_ttc, on='Date', how='left')
df_merged = df_merged.rename(columns={"Date": "Date_monthly"})
print("df_merged\n", df_merged)

os.makedirs("csv", exist_ok=True)
df_merged.to_csv("csv/gazole_htt_ttc_eur_liter_monthly.csv", index=False)

df_carburants_htt
           Date  gazole_eur_liter_htt
0   2015-01-01                0.4484
1   2015-02-01                0.5081
2   2015-03-01                0.5251
3   2015-04-01                0.5205
4   2015-05-01                0.5477
..         ...                   ...
128 2025-09-01                0.7298
129 2025-10-01                0.7247
130 2025-11-01                0.7854
131 2025-12-01                0.7081
132 2026-01-01                0.7544

[133 rows x 2 columns]
df_carburants_ttc
           Date  gazole_eur_liter_ttc
0   2015-01-01                1.1149
1   2015-02-01                1.1865
2   2015-03-01                1.2069
3   2015-04-01                1.2014
4   2015-05-01                1.2341
..         ...                   ...
128 2025-09-01                1.6067
129 2025-10-01                1.6006
130 2025-11-01                1.6735
131 2025-12-01                1.5808
132 2026-01-01                1.6344

[133 rows x 2 columns]
df_merged
     Date_monthl